In [49]:
#El objetivo de este código es hacer scraping de la página del Boletín oficial de la República Argentina
#Luego, se utiliza una librería de procesamiento de lenguaje natural para realizar un simple análisis inicial

In [13]:
#Para el scraping se utiliza la librería Selenium
#Selenium es un conjunto de herramientas de software de código abierto utilizado para la 
#automatización de pruebas funcionales en aplicaciones web. Selenium proporciona una 
#interfaz de programación de aplicaciones (API) para controlar un navegador web y simular 
#la interacción humana con una página web.

#Importar librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

#Se crean varias Listas vacías
poderes = []
normas = []
fechas = []
descripciones = []
tema_buscado = []
año_buscado = []
            
# configurar el webdriver
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)
# abrir la página web
driver.get("https://www.boletinoficial.gob.ar/busquedaAvanzada/primera")

#Definir temas a buscar (en el campo palabra clave). La idea es buscar por tema, sin definir tipo de norma
temas= ['dolar',
        'tipo de cambio',
        'mercado cambiario',
        'devaluación',
        'moneda extranjera'
       ]

#Definir años de búsqueda
año_inicio = 2003
año_fin = 2016
años = list(range(año_inicio, año_fin))
años = list(map(str, años))


#iterar sobre los temas
for j in temas:
    #Iterar sobre los años
    for i in años:
    
        # Completar el campo de búsqueda palabra clave
        search_box1 = driver.find_element(By.ID, "palabraClave")
        search_box1.send_keys(j)

        # Completar el campo de búsqueda año
        search_box2 = driver.find_element(By.ID, "anioNormaIP")
        search_box2.send_keys(i)
     
        #Intentar realizar la búsqueda, si no encuentra elementos para una combinación dada de tema/año, pasa a la siguiente
        try:
            # hacer clic en el botón de búsqueda
            search_button = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "btnBusquedaAvanzada")))
            search_button.click()
            # espera a que se carguen nuevos resultados (1 segundo)
            time.sleep(1)
            # limpiar ambos campos de entrada
            search_box1.clear()
            search_box2.clear()
                       
            # hacer scroll hasta el final de la página
            while True:
                # obtener número actual de resultados
                num_resultados = len(driver.find_elements(By.XPATH, "//div[contains(@class, 'linea-aviso')]"))
                # hacer scroll hasta el final de la página
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                # esperar a que se carguen nuevos resultados (2 segundos)
                time.sleep(2)
                # obtener nuevo número de resultados
                num_nuevos_resultados = len(driver.find_elements(By.XPATH, "//div[contains(@class, 'linea-aviso')]"))
                # salir del bucle si no hay más resultados nuevos
                if num_nuevos_resultados == num_resultados:
                    break

            # extraer resultados visibles
            # Obtener todos los bloques de información de cada tema/año
            bloques = driver.find_elements(By.XPATH, "//div[contains(@class, 'linea-aviso')]")
            
            # Iterar sobre los bloques y separar los datos
            #se extraen las class ="item", que contiene sólo un elemento, y las class="item-detalle", que son tres
            for bloque in bloques:
                #Se arman listas con cada componente:
                # Obtener el poder del que emana la norma
                poder = bloque.find_element(By.XPATH, ".//p[contains(@class, 'item')]").text
                poderes.append(poder)
                # Obtener la norma (númeor y año)
                norma = bloque.find_elements(By.XPATH, ".//p[contains(@class, 'item-detalle')]")[0].text
                normas.append(norma)
                # Obtener la Fecha de Publicación
                fecha_publicacion = bloque.find_elements(By.XPATH, ".//p[contains(@class, 'item-detalle')]")[1].text
                fechas.append(fecha_publicacion)
                # Obtener la descripción
                descripcion = bloque.find_elements(By.XPATH, ".//p[contains(@class, 'item-detalle')]")[2].text
                descripciones.append(descripcion)
                #registrar el tema que se buscó
                tema_buscado.append(j)
                #registrar el año que se buscó
                año_buscado.append(i)
                          
        except:
            # si no se encuentra la palabra clave, pasar a la siguiente iteración sin generar un mensaje de error
            pass
      

In [33]:
#crear el dataframe
df = pd.DataFrame({'Poder':poderes, 'Norma':normas, 'Fecha':fechas, 'Descripción':descripciones, 'tema_buscado':tema_buscado, 'año_buscado':año_buscado})        

df = df.drop_duplicates(subset=['Norma'])

#mejorar algunas columnas
df[['Norma', 'Año']] = df['Norma'].str.split('/', n=1, expand=True)
df[['Norma', 'Número']] = df['Norma'].str.split(' ', n=1, expand=True)
df['Fecha'] = df['Fecha'].str.replace('Fecha de Publicacion: ', '')

In [34]:
df

,Poder,Norma,Fecha,Descripción,tema_buscado,año_buscado,Año,Número
0,PODER LEGISLATIVO,Ley,04/12/2003,Modifícase la Ley Nº 25.561. Prorrógase la dec...,dolar,2003,2003,25820
1,PODER EJECUTIVO,Decreto,18/12/2003,"Danse por prorrogadas, a partir del 21 de octu...",dolar,2003,2003,1251
2,PODER EJECUTIVO,Decreto,18/12/2003,Desígnase Subsecretaria de Coordinación de la ...,dolar,2003,2003,1262
3,PODER EJECUTIVO,Decreto,18/12/2003,"Dase por aceptada, a partir del 10 de diciembr...",dolar,2003,2003,1261
4,PODER EJECUTIVO,Decreto,18/12/2003,"Establécese, para el Programa de Financiamient...",dolar,2003,2003,1274
...,...,...,...,...,...,...,...,...
3297,MINISTERIO DE DESARROLLO SOCIAL SUBSECRETARÍA ...,Disposición,19/05/2015,con identificación de la presente contratación...,moneda extranjera,2015,2015,75
3298,MINISTERIO DE DESARROLLO SOCIAL SUBSECRETARÍA ...,Disposición,19/05/2015,con identificación de la presente contratación...,moneda extranjera,2015,2015,66
3299,MINISTERIO DE DESARROLLO SOCIAL SUBSECRETARÍA ...,Disposición,15/05/2015,con identificación de la presente contratación...,moneda extranjera,2015,2015,74
3300,MINISTERIO DE DESARROLLO SOCIAL SUBSECRETARÍA ...,Disposición,15/05/2015,con identificación de la presente contratación...,moneda extranjera,2015,2015,68


In [ ]:
#ahora se realiza el procesamiento de lenguaje natural
#para ello se utiliza la librería Spacy

In [16]:
#Existen otras como NLTK o Pattern.
#Spacy es una biblioteca más reciente que NLTK y está diseñada específicamente para ser rápida y eficiente.

#El primer paso es Tokenizar 
#cuyo objetivo es generar una columna con el texto dividido en unidades más pequeñas llamadas tokens. 
#Los tokens son una secuencia de caracteres que representan una palabra o un símbolo en un texto.

#importar la librería
import spacy

# Cargar el modelo de español en spaCy
nlp = spacy.load('es_core_news_sm')
#Definir una función para tokenizar
def tokenizar(text):
    doc = nlp(text)
    return [token.text for token in doc]
#aplicar la función
df['tokens'] = df['Descripción'].apply(tokenizar)

#En un segundo paso se puede lemmatizar
#Esto es para normalizar el texto, transformando las palabras en su base o raíz
# Definir una función para lematizar una palabra
def lemmatizar(word):
    lemma = nlp(word)[0].lemma_
    return lemma
# aplicar la función
df['texto_lemmatized'] = df['Descripción'].apply(lambda x: ' '.join([lemmatizar(word) for word in nltk.word_tokenize(x, language='spanish')]))

In [21]:
#Por último, podemos hacer un análisis adicional, separando entidades gramaticales
#(sustantivos, verbos, edjetivos, etc)
#Esto puede falicitar mucho el análisis de las normas, buscando características específicas

# Definir función para analizar texto
def analizar_texto(texto):
    doc = nlp(texto)
    #usar el modelo para extraer diversas entidades del resumen del contenido de cada norma 
    #Obtener sustantivos y nombres propios, verbos y adjetivos
    sustantivos_y_nombres_propios = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    verbos = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    adjetivos = [token.text for token in doc if token.pos_ == 'ADJ']
    # Encontrar caracteres numéricos 
    #(que normalmente representan otras normas a las que hace referencia la que estamos analizando)
    numeros = [token.text for token in doc if token.is_digit]
    return sustantivos_y_nombres_propios, verbos, adjetivos, numeros

#crear 4 columnas nuevas con las distintas entidades aplicando la función a la columna 'Descripción' 
df['Descripción'].fillna('', inplace=True)
df[['sustantivos', 'verbos', 'adjetivos', 'numeros']] = pd.DataFrame(df['Descripción'].apply(analizar_texto).tolist())

In [27]:
#guardar dataset con información categorizada de normas de diversos temas y 
#con la amplitud de fechas que queramos

df.to_excel('normas nuevas.xlsx', index=False)



## FIN